In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import time
import numpy as np
from PIL import Image
from torch.utils.data.dataset import Dataset
from tqdm import tqdm
from torchvision import transforms
from torchvision import models
import torch
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import precision_score, recall_score, f1_score
from torch import nn
from torch.utils.data.dataloader import DataLoader
from matplotlib import pyplot as plt
from numpy import printoptions
import requests
import tarfile
import random
import json
from shutil import copyfile

In [ ]:
torch.manual_seed(2020)
torch.cuda.manual_seed(2020)
np.random.seed(2020)
random.seed(2020)
torch.backends.cudnn.deterministic = True

In [ ]:
import os, sys, tarfile
from tqdm import tqdm
def extract(tar_url, extract_path='.'):
    print(tar_url)
    tar = tarfile.open(tar_url, 'r')
    for item in tqdm(tar):
        tar.extract(item, extract_path)
        if item.name.find(".tgz") != -1 or item.name.find(".tar") != -1:
            extract(item.name, "./" + item.name[:item.name.rfind('/')])

In [ ]:
try:
    extract('../input/yelp-restaurant-photo-classification/train_photos' + '.tgz')
    print ('Done.')
except:
    #name = os.path.basename(sys.argv[0])
    print('error')

In [ ]:
from tqdm import tqdm
try:
    extract('../input/yelp-restaurant-photo-classification/test_photos.tgz')
    print ('Done.')
except:
    #name = os.path.basename(sys.argv[0])
    print('error')

In [ ]:
from tqdm import tqdm
try:
    extract('../input/yelp-restaurant-photo-classification/train.csv.tgz')
    extract('../input/yelp-restaurant-photo-classification/train_photo_to_biz_ids.csv.tgz')
    extract('../input/yelp-restaurant-photo-classification/test_photo_to_biz.csv.tgz')
    extract('../input/yelp-restaurant-photo-classification/sample_submission.csv.tgz')
    print ('Done.')
except:
    #name = os.path.basename(sys.argv[0])
    print('error')

In [ ]:
import pandas as pd
train_biz=pd.read_csv('./train_photo_to_biz_ids.csv')
train_biz=train_biz.head(5000)
test_biz_biz=pd.read_csv('./test_photo_to_biz.csv')
test_biz_biz=test_biz_biz.head(1000)
train=pd.read_csv('./train.csv')
train=train.head(1000)
sub=pd.read_csv('./sample_submission.csv')
train_biz.head()

In [ ]:
data=pd.merge(train_biz,train, on='business_id',how='left')
data=data.head(5000)


In [ ]:
data_test=pd.merge(test_biz_biz,sub, on='business_id',how='left') 
data_test.head()


In [ ]:
data=data.dropna(subset=['labels'])
data.head()

In [ ]:
data['labs']=data['labels'].apply(lambda x:str(x).split(' '))
data

In [ ]:
labelsint=data.labs.tolist()
for i in tqdm(labelsint):
    for j in range(len(i)):
        #print(j)
        #break
        i[j]=int(i[j])
    

In [ ]:
data['labsint']=labelsint
data

In [ ]:
from sklearn import metrics, model_selection, preprocessing
df_train, df_valid = model_selection.train_test_split(
        data, test_size=0.1, random_state=42
)

In [ ]:
class NusDataset(Dataset):
    def __init__(self, data_path, data, transforms):
        self.transforms = transforms
        data=data
        samples = data['photo_id'].tolist()
        labs=data['labsint'].tolist()
        self.classes = [0,1,2,3,4,5,6,7,8]

        self.imgs = []
        self.annos = []
        self.data_path = data_path
        #print('loading', anno_path)
        for sample in samples:
            self.imgs.append(sample)
        for lab in labs:
            self.annos.append(lab)
            
        for item_id in range(len(self.annos)):
            item = self.annos[item_id]
            vector = [cls in item for cls in self.classes]
            self.annos[item_id] = np.array(vector, dtype=float)

    def __getitem__(self, item):
        anno = self.annos[item]
        img_path = os.path.join(self.data_path, str(self.imgs[item])+'.jpg')
        img = Image.open(img_path)
        if self.transforms is not None:
            img = self.transforms(img)
        return img, anno

        

    def __len__(self):
        return len(self.imgs)

In [ ]:
dataset_val = NusDataset('./train_photos', df_valid, None)
dataset_train = NusDataset('./train_photos', df_train, None)


class Resnext50(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        resnet = models.resnext50_32x4d(pretrained=True)
        resnet.fc = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=resnet.fc.in_features, out_features=n_classes)
        )
        self.base_model = resnet
        self.sigm = nn.Sigmoid()

    def forward(self, x):
        return self.sigm(self.base_model(x))

In [ ]:
def calculate_metrics(pred, target, threshold=0.5):
    pred = np.array(pred > threshold, dtype=float)
    return {'micro/precision': precision_score(y_true=target, y_pred=pred, average='micro'),
            'micro/recall': recall_score(y_true=target, y_pred=pred, average='micro'),
            'micro/f1': f1_score(y_true=target, y_pred=pred, average='micro'),
            'macro/precision': precision_score(y_true=target, y_pred=pred, average='macro'),
            'macro/recall': recall_score(y_true=target, y_pred=pred, average='macro'),
            'macro/f1': f1_score(y_true=target, y_pred=pred, average='macro'),
            'samples/precision': precision_score(y_true=target, y_pred=pred, average='samples'),
            'samples/recall': recall_score(y_true=target, y_pred=pred, average='samples'),
            'samples/f1': f1_score(y_true=target, y_pred=pred, average='samples'),
            }

In [ ]:

# Initialize the training parameters.
num_workers = 8 
lr = 1e-4 # Learning rate
batch_size = 32
save_freq = 35 # checkpoint frequency (epochs)
test_freq = 200 # Test model frequency (iterations)
max_epoch_number = 15 # Number of epochs for training 


mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

device = torch.device('cuda')
# Save path for checkpoints
save_path = 'chekpoints/'
# Save path for logs
logdir = 'logs/'



In [ ]:
def checkpoint_save(model, save_path, epoch):
    f = os.path.join(save_path, 'checkpoint-{:06d}.pth'.format(epoch))
    if 'module' in dir(model):
        torch.save(model.module.state_dict(), f)
    else:
        torch.save(model.state_dict(), f)
    print('saved checkpoint:', f)

In [ ]:
val_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
print(tuple(np.array(np.array(mean)*255).tolist()))

# Train preprocessing
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(),
    transforms.RandomAffine(degrees=20, translate=(0.2, 0.2), scale=(0.5, 1.5),
                            shear=None, resample=False, 
                            fillcolor=tuple(np.array(np.array(mean)*255).astype(int).tolist())),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [ ]:
#test_annotations = os.path.join(img_folder, 'small_test.json')
#train_annotations = os.path.join(img_folder, 'small_train.json')

test_dataset = NusDataset('./train_photos', df_valid, val_transform)
train_dataset = NusDataset('./train_photos', df_train, train_transform)



train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True,
                              drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers)

num_train_batches = int(np.ceil(len(train_dataset) / batch_size))

# Initialize the model
model = Resnext50(len(train_dataset.classes))
# Switch model to the training mode and move it to GPU.
model.train()
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

os.makedirs(save_path, exist_ok=True)

# Loss function
criterion = nn.BCELoss()
# Tensoboard logger
logger = SummaryWriter(logdir)

In [ ]:
epoch = 0
iteration = 0
best = [0,0,0]
while True:
    batch_losses = []
    for imgs, targets in tqdm(train_dataloader):
        imgs, targets = imgs.to(device), targets.to(device)

        optimizer.zero_grad()

        model_result = model(imgs)
        loss = criterion(model_result, targets.type(torch.float))

        batch_loss_value = loss.item()
        loss.backward()
        optimizer.step()

        logger.add_scalar('train_loss', batch_loss_value, iteration)
        batch_losses.append(batch_loss_value)
        with torch.no_grad():
            result = calculate_metrics(model_result.cpu().numpy(), targets.cpu().numpy())
            for metric in result:
                logger.add_scalar('train/' + metric, result[metric], iteration)

        if iteration % test_freq == 0:
            model.eval()
            with torch.no_grad():
                model_result = []
                targets = []
                for imgs, batch_targets in tqdm(test_dataloader):
                    imgs = imgs.to(device)
                    model_batch_result = model(imgs)
                    model_result.extend(model_batch_result.cpu().numpy())
                    targets.extend(batch_targets.cpu().numpy())

            result = calculate_metrics(np.array(model_result), np.array(targets))
            for metric in result:
                logger.add_scalar('test/' + metric, result[metric], iteration)
            print("epoch:{:2d} iter:{:3d} test: "
                  "micro f1: {:.3f} "
                  "macro f1: {:.3f} "
                  "samples f1: {:.3f}".format(epoch, iteration,
                                              result['micro/f1'],
                                              result['macro/f1'],
                                              result['samples/f1']))
            
            best[0]= max([best[0],result['micro/f1']])
            best[1]= max([best[1],result['macro/f1']])
            best[2]= max([best[2],result['samples/f1']])
            model.train()
        iteration += 1

    loss_value = np.mean(batch_losses)
    print("epoch:{:2d} iter:{:3d} train: loss:{:.3f}".format(epoch, iteration, loss_value))
    if epoch % save_freq == 0:
        checkpoint_save(model, save_path, epoch)
    epoch += 1
    if max_epoch_number < epoch:
        break

In [ ]:
print("Micro F1 : "+str(best[0])+"  Macro F1 : "+str(best[1])+ "     Samples F1 : "+str(best[2]))

model.eval()
label_notation = {0: 'good_for_lunch', 1: 'good_for_dinner', 2: 'takes_reservations',  3: 'outdoor_seating',
                  4: 'restaurant_is_expensive', 5: 'has_alcohol', 6: 'has_table_service', 7: 'ambience_is_classy',
                  8: 'good_for_kids'}
if torch.cuda.is_available():
    model.cuda()
for sample_id in [1,2,3,4,6,7,8,9,10,11]:
    test_img, test_labels = test_dataset[sample_id]
    print(type(test_img))
    test_img = test_img.cuda()
    test_img_path = os.path.join('./train_photos', str(test_dataset.imgs[sample_id])+'.jpg')
    with torch.no_grad():
        raw_pred = model(test_img.unsqueeze(0)).cpu().numpy()[0]
        raw_pred = np.array(raw_pred > 0.5, dtype=float)

    predicted_labels = np.array(dataset_val.classes)[np.argwhere(raw_pred > 0)[:, 0]]
    if not len(predicted_labels):
        predicted_labels = ['no predictions']
    img_labels = np.array(dataset_val.classes)[np.argwhere(test_labels > 0)[:, 0]]
    
    result = [label_notation[p] for p in predicted_labels]
    expected = [label_notation[p] for p in img_labels]
    plt.imshow(Image.open(test_img_path))
    print(result)
    print(expected)
    plt.axis('off')
    plt.show()

In [ ]:

    test_img_path = '../input/lablebi/sandwich.jpg'
    model.eval()
    with torch.no_grad():
        model_result = []
        targets = []
        test_img = Image.open(test_img_path)
        test_img = val_transform(test_img)
        test_img = test_img.to(device)
        raw_pred = model(test_img.unsqueeze(0)).cpu().numpy()[0]
        

    predicted_labels = np.array(dataset_val.classes)[np.argwhere(raw_pred > 0.5)[:, 0]]

    
    result = [label_notation[p] for p in predicted_labels]
    plt.imshow(Image.open(test_img_path))
    print(result)
    plt.axis('off')
    plt.show()

In [ ]:
    test_img_path = '../input/lablebi/pizza.jpg'
    model.eval()
    with torch.no_grad():
        model_result = []
        targets = []
        test_img = Image.open(test_img_path)
        test_img = val_transform(test_img)
        test_img = test_img.to(device)
        raw_pred = model(test_img.unsqueeze(0)).cpu().numpy()[0]
        

    predicted_labels = np.array(dataset_val.classes)[np.argwhere(raw_pred > 0.5)[:, 0]]

    
    result = [label_notation[p] for p in predicted_labels]
    plt.imshow(Image.open(test_img_path))
    print(result)
    plt.axis('off')
    plt.show()

In [ ]:
    test_img_path = '../input/lablebi/couscous.jpg'
    model.eval()
    with torch.no_grad():
        model_result = []
        targets = []
        test_img = Image.open(test_img_path)
        test_img = val_transform(test_img)
        test_img = test_img.to(device)
        raw_pred = model(test_img.unsqueeze(0)).cpu().numpy()[0]
        

    predicted_labels = np.array(dataset_val.classes)[np.argwhere(raw_pred > 0.5)[:, 0]]

    
    result = [label_notation[p] for p in predicted_labels]
    plt.imshow(Image.open(test_img_path))
    print(result)
    plt.axis('off')
    plt.show()

In [ ]:
    test_img_path = '../input/lablebi/mechwi.jpg'
    model.eval()
    with torch.no_grad():
        model_result = []
        targets = []
        test_img = Image.open(test_img_path)
        test_img = val_transform(test_img)
        test_img = test_img.to(device)
        raw_pred = model(test_img.unsqueeze(0)).cpu().numpy()[0]
        

    predicted_labels = np.array(dataset_val.classes)[np.argwhere(raw_pred > 0.5)[:, 0]]

    
    result = [label_notation[p] for p in predicted_labels]
    plt.imshow(Image.open(test_img_path))
    print(result)
    plt.axis('off')
    plt.show()